In [4]:
import pandas as pd
import numpy as np

traindf = pd.read_csv('data/train.csv')
testdf = pd.read_csv('data/test.csv')
traindf['Set'] = 'Train'
testdf['Set'] = 'Test'
df = pd.concat([testdf,traindf])


In [5]:
len(df.Cabin.unique())

9826

In [6]:
def splits(data_frame):
    split_df = data_frame['PassengerId'].str.split('_', expand=True)
    data_frame[['Group', 'GroupNumber']] = split_df

    split_df =  data_frame['Cabin'].str.split("/", expand = True)
    data_frame[['Deck', 'Number', 'Side']]= split_df
    data_frame.Number = data_frame.Number.astype('Int64')
    
    split_df = data_frame['Name'].str.split(" ",expand = True)
    data_frame[['FirstName','LastName']] = split_df


    return data_frame

df = splits(df)


In [7]:
df[(df.Deck == 'T') & (df.Side == 'P')]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Name,Set,Transported,Group,GroupNumber,Deck,Number,Side,FirstName,LastName
1494,3208_01,Europa,False,T/4/P,TRAPPIST-1e,27.0,False,0.0,11293.0,0.0,...,Thabius Reeddommy,Test,NaN,3208,01,T,4,P,Thabius,Reeddommy
1004,1071_01,NaN,False,T/0/P,TRAPPIST-1e,35.0,False,415.0,1328.0,0.0,...,Alraida Dingauge,Train,False,1071,01,T,0,P,Alraida,Dingauge
2254,2414_01,Europa,False,T/1/P,TRAPPIST-1e,42.0,False,0.0,1829.0,2.0,...,Mergak Headfair,Train,False,2414,01,T,1,P,Mergak,Headfair
2734,2935_01,Europa,False,T/2/P,TRAPPIST-1e,33.0,False,0.0,28.0,0.0,...,NaN,Train,False,2935,01,T,2,P,NaN,NaN
2763,2971_01,Europa,False,T/3/P,TRAPPIST-1e,38.0,False,0.0,3135.0,0.0,...,Pent Sessiouse,Train,True,2971,01,T,3,P,Pent,Sessiouse


In [8]:
a = 0
for deck in ['A','B','C','D','E','F','G']:
    for side in ['P','S']:
        a += max(df[(df.Deck == deck) & (df.Side == side)].Number) + 1
a

10005

In [9]:
def group_size(df):
    def group_size_apply(row):
        group = df[df.Group == row.Group]
        return len(group)
    df['GroupSize'] = df.apply(group_size_apply, axis = 1)
    return df
df = group_size(df)

In [10]:
df['Bills'] = df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck']
df.loc[(df['Age'] < 13), 'Bills'] = 0
df.loc[(df['CryoSleep'] == True),'Bills'] = 0 


In [11]:
def impute_attribute_based_on_shared_feature(df, attribute, feature):
    # Function to fill NaNs in a group based on the first available non-NaN value
    def fill_with_first_available(group_df):
        if not group_df[attribute].dropna().empty:
            first_available = group_df[attribute].dropna().iloc[0]
            group_df[attribute] = group_df[attribute].fillna(first_available)
        return group_df

    # Filter out rows where the feature is NaN, then apply the fill function to each group
    filtered_df = df.dropna(subset=[feature])
    imputed_df = filtered_df.groupby(feature, group_keys=False).apply(fill_with_first_available)
    
    # Merge back with original df rows where the feature is NaN to retain those rows unchanged
    df_with_nan_feature = df[df[feature].isna()]
    final_df = pd.concat([imputed_df, df_with_nan_feature], ignore_index=True)

    return final_df

In [12]:
df = impute_attribute_based_on_shared_feature(df,'HomePlanet','Group')

/var/folders/nk/n25bjxk97j7213hk3kmrl6bm0000gp/T/ipykernel_44784/2494772570.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  imputed_df = filtered_df.groupby(feature, group_keys=False).apply(fill_with_first_available)


In [13]:
df = impute_attribute_based_on_shared_feature(df,'HomePlanet','LastName')

/var/folders/nk/n25bjxk97j7213hk3kmrl6bm0000gp/T/ipykernel_44784/2494772570.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  imputed_df = filtered_df.groupby(feature, group_keys=False).apply(fill_with_first_available)


In [14]:
df = df.sort_values(by = ['Group','GroupNumber'])
df = df.reset_index(drop = True)


In [15]:
decks_by_planet = {
    'Earth':['E','F','G'],
    'Europa': ['A','B','C','D','E','T'],
    'Mars': ['D','E','F']
}

decks_by_planet_no_bills = {
    'Earth':['G'],
    'Europa':['B'],
    'Mars': ['E','F']
}

planet_by_deck = {
    'A':['Europa'],'B':['Europa'],'C':['Europa'],'D':['Europa','Mars'],'E':['Europa','Mars','Earth'],
    'F':['Earth','Mars'],'G':['Earth'],'T':['Europa']
}

In [18]:
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,Group,GroupNumber,Deck,Number,Side,FirstName,LastName,GroupSize,Bills
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,False,0001,01,B,0,P,Maham,Ofracculy,1,0.0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,True,0002,01,F,0,S,Juanna,Vines,1,736.0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,False,0003,01,A,0,S,Altark,Susent,2,10383.0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,...,False,0003,02,A,0,S,Solam,Susent,2,5176.0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,...,True,0004,01,F,1,S,Willy,Santantines,1,1091.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,9277_01,Earth,True,G/1498/S,PSO J318.5-22,43.0,False,0.0,0.0,0.0,...,NaN,9277,01,G,1498,S,Lilace,Leonzaley,1,0.0
12966,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,...,False,9278,01,G,1499,S,Kurta,Mondalley,1,0.0
12967,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,...,True,9279,01,G,1500,S,Fayey,Connon,1,1873.0
12968,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,...,False,9280,01,E,608,S,Celeon,Hontichre,2,4637.0


In [19]:
df.to_csv('pre_processed.csv',index = False)